In [2]:
import pandas as pd

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [4]:
# Agrupaciones

# Zona Provincias
provincias_sur = ['Yucatán','Quintana Roo','Campeche','Tabasco', 'Veracruz', 'Oaxaca', 'Chiapas', 'Guerrero', 'Michoacán']
provincias_centro = ['Tlaxcala','Morelos','Puebla','Hidalgo','Aguascalientes', 'Distrito Federal', 'Jalisco', 'Edo. de México','Querétaro', 'Colima', 'Nayarit', 'Aguascalientes', 'Guanajuato', 'Hidalgo']
provincias_norte = ['Sonora','San luis Potosí','Baja California Norte', 'Baja California Sur', 'Chihuahua', 'Durango', 'Coahuila', 'Sinaloa', 'Nuevo León',  'Tamaulipas']

# Intervalos metros
punto20 = df_train['metrostotales'].quantile(0.2)
punto40 = df_train['metrostotales'].quantile(0.4)
punto60 = df_train['metrostotales'].quantile(0.6)
punto80 = df_train['metrostotales'].quantile(0.8)
punto20_cubiertos = df_train['metroscubiertos'].quantile(0.2)
punto40_cubiertos = df_train['metroscubiertos'].quantile(0.4)
punto60_cubiertos = df_train['metroscubiertos'].quantile(0.6)
punto80_cubiertos = df_train['metroscubiertos'].quantile(0.8)

In [17]:
def features_independientes_precio(df):
    '''Devuelve una copia del dataframe con nuevas columnas, 
    donde los datos generados no dependen del precio'''
    df = df.copy()
    df['escomercial'] = df.tipodepropiedad.str.contains('.*omercial.*').astype(bool)
    df['porcentaje_metros'] = df['metroscubiertos']/df['metrostotales']
    df['zona'] = list(map(lambda x: 'Norte ' if x in provincias_norte else ('Sur' if x in provincias_sur else 'Centro'), df['provincia']))
    df['intervalo_metros_totales'] = list(map(lambda x: 'Grupo1' if x<punto20 else ('Grupo2' if punto20<x<punto40 else ('Grupo3' if punto40<x<punto60 else('Grupo4' if punto60<x<punto80 else 'Grupo5'))), df['metrostotales']))
    df['intervalo_metros_cubiertos'] = list(map(lambda x: 'Grupo1' if x<punto20_cubiertos else ('Grupo2' if punto20_cubiertos<x<punto40_cubiertos else ('Grupo3' if punto40_cubiertos<x<punto60_cubiertos else('Grupo4' if punto60_cubiertos<x<punto80_cubiertos else 'Grupo5'))), df['metrostotales']))
    df['promedio_metros_tipo_propiedad'] = df.groupby(['tipodepropiedad'])['metrostotales'].transform('mean')
    df['promedio_metros_tipo_propiedad'] = df.groupby(['tipodepropiedad'])['metroscubiertos'].transform('mean')
    
    return df

def features_dependientes_precio(df, df_train):
    '''Devuelve una copia del dataframe `df` con nuevas columnas dependientes del precio,
    usando el dataframe `df_train` con los precios para generar los datos.
    El dataframe `df` puede ser el mismo que `df_train`.'''
    df = df.copy()
    df_train = df_train.copy()
    
    df_train['promedio_precio_ciudad'] = df_train.groupby(['provincia', 'ciudad'])['precio'].transform('mean')
    df_promedios = df_train.groupby(['provincia', 'ciudad']).agg(promedio_precio_ciudad=('precio', 'mean')).reset_index()
    df = pd.merge(df, df_promedios, on=['provincia', 'ciudad'], how='left')
    df['promedio_precio_ciudad'] = df['promedio_precio_ciudad'].fillna(df_train['promedio_precio_ciudad'].mean())
    
    df_train['promedio_id_zona'] = df_train.groupby(['idzona'])['precio'].transform('mean')
    df_promedio = df_train.groupby('idzona').agg(promedio_id_zona=('precio', 'mean'))
    df = pd.merge(df, df_promedio, on='idzona', how='left')
    df['promedio_id_zona'] = df['promedio_id_zona'].fillna(df['promedio_precio_ciudad'])
    
    return df